In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## Creating subagents

In [ ]:
from langchain.tools import tool


@tool
def square_root(x: float) -> float:
    """Calculate the square root of a number"""
    return x**0.5


@tool
def square(x: float) -> float:
    """Calculate the square of a number"""
    return x**2

In [ ]:
from langchain.agents import create_agent

# create subagents

subagent_1 = create_agent(model="gpt-5-nano", tools=[square_root])

subagent_2 = create_agent(model="gpt-5-nano", tools=[square])

## Calling subagents

In [ ]:
from langchain.messages import HumanMessage


@tool
def call_subagent_1(x: float) -> float:
    """Call subagent 1 in order to calculate the square root of a number"""
    response = subagent_1.invoke(
        {"messages": [HumanMessage(content=f"Calculate the square root of {x}")]}
    )
    return response["messages"][-1].content


@tool
def call_subagent_2(x: float) -> float:
    """Call subagent 2 in order to calculate the square of a number"""
    response = subagent_2.invoke(
        {"messages": [HumanMessage(content=f"Calculate the square of {x}")]}
    )
    return response["messages"][-1].content


## Creating the main agent

main_agent = create_agent(
    model="gpt-5-nano",
    tools=[call_subagent_1, call_subagent_2],
    system_prompt="You are a helpful assistant who can call subagents to calculate the square root or square of a number.",
)

## Test

In [5]:
question = "What is the square root of 456?"

response = main_agent.invoke({"messages": [HumanMessage(content=question)]})

In [6]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='What is the square root of 456?', additional_kwargs={}, response_metadata={}, id='4c57fdaa-3d32-4e63-8217-1b1fec8e934a'),
              AIMessage(content='sqrt(456) = 2 * sqrt(114) ≈ 21.354.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1628, 'prompt_tokens': 202, 'total_tokens': 1830, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 1600, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D8Z8oFcNOuEmQZvvFAkgUvgKj6TBS', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c53e5-8c12-7ba3-8cfc-8e578cddd499-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 202, 'output_tokens': 1628, 'total_tokens': 1830, 'input_token_details': {'audio':